# Bayesian Networks

In [58]:
import math 
from pomegranate import *
import json

In [59]:
def addNewTable(node, distribution, conditional):
    
    ##List that contain the distributions and conditionals of the Conditional Probability Table
    conditional_list = []
    distribution_list = []
    
    ##Reads every k in the node
    for k in node.keys():
        
        ##List that contain the probability table of the Conditional Probability Table
        probability_list = []
        
        ##Info has the informations of the name and parents of the state 
        info = k.split("|")
        
        ##Parents has the list of parents of the state 
        parents = info[1].split(",")
        
        ##For every parent of the k it will push it to the probability list
        for i in range(len(parents)):
            probability_list.append(parents[i])
         
        ##It pushes the numerical value of the probability 
        probability_list.append(node.get(k))
        
        ##Adds that probability to the table
        conditional_list.append(probability_list)
   
    ##Reads every k in the node
    for k in node.keys():
        
        ##Info has the informations of the name and parents of the state 
        info = k.split("|")
        
        ##Parents has the list of parents of the state 
        parents = info[1].split(",")
        
        ##For every parent of the k it will push it to the distribution list
        for i in range(len(parents)):
            
            ##For every distribution that equals to the parents  will push it to the distribution list
            for d in range(len(distribution)):
                if parents[i] == distribution[d].keys()[0] or parents[i] == "-"+distribution[d].keys()[0]:
                    distribution_list.append(distribution[d])
                    break 
        break   
    
    ##Creates a Conditional Probability Table with the conditional_list and distribution_list generated in the function
    cond = ConditionalProbabilityTable(conditional_list,distribution_list)
    
    ##Returns the Conditional Probability Table cond
    return cond    
    


            

In [60]:
with open("Noisy_OR.json", "r") as f:
    bayer_data = json.load(f)

##Lists that contain the distributions, conditionals and states of the Bayesian Network
distribution_list = []
state_list = []
conditional_list = []

counter = 0


##Reads every line of the JSON file 
for node in bayer_data:
    
    ##Reads every key value from the node that is reading at the moment
    for k in node.keys():
        
        ##If the k does not have a character "|" it means is a Discrete Distribution
        if k.find("|") == -1:
            
            ##Creates a Discrete Distribution with the k value as the name and the information of the key as the rest
            distribution = DiscreteDistribution({k : node.get(k)})
            
            ##Push the distribution to the distribution_list
            distribution_list.append(distribution)
            
            ##Creates a new state with the distribution and the name k
            state_list.append(State(distribution, name = k))
            
        ##If the k has a "|" it means is a Conditional Probability Table    
        else:
            
            ##Calls the function addNewTable that returns a Conditional Probability Table
            cond = addNewTable(node, distribution_list, conditional_list)
            
            ##Push the cond to the conditional_list 
            conditional_list.append(cond)
            
            ##Creates a new state with the conditional and the name k
            state_list.append(State(cond, name = k.split("|")[0]))
            
            break

            
network = BayesianNetwork("Name")
network.add_state(state_list)
